Import libraries

In [1]:
# import os
# import sys
# import datetime
import glob as glob
import numpy as np
import cv2
import pandas as pd
import keras
from keras.applications.inception_v3 import preprocess_input, decode_predictions
# from keras.applications.vgg16 import VGG16, preprocess_input
# from keras.models import Model
# from keras.layers import Dense, GlobalAveragePooling2D
# from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
# from keras.optimizers import SGD
# import tensorflow
# from scipy.interpolate import spline

# import matplotlib.pyplot as plt
# %matplotlib inline

Using TensorFlow backend.


Import the pretrained model

In [2]:
from keras.applications import InceptionV3

Instatiate a copy of the model and copy in the labels

In [3]:
model = InceptionV3()
labels = pd.read_csv('/Users/tasanders/Documents/TF_Test_Project/categories.csv')


Names of the folders to investigate. Note that this assumes we're in the /Users/tasanders/Documents/TF_Test_Project/ folder. Should probably make this more general.

In [4]:
folders = ['With Screens', 'Without Screens']
locations = ['Brinno Camera Test', 'Photos to test Inception']

Make a function to create a CSV for each folder with the probabilities and labels.

In [5]:
def predict_and_save(folder,location):
    num_images = len(glob.glob(location+'/'+folder+'/*.jpg'))
    predict_files = glob.glob(location+'/'+folder+'/*.jpg')

    im = cv2.imread(predict_files[0])
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = cv2.resize(im, (256, 256)).astype(np.float32)
    im = np.expand_dims(im, axis = 0)/255
    
    predictor, prob, image_id = [], [], []
    for i in predict_files:
        im = cv2.imread(i)
        im = cv2.resize(cv2.cvtColor(im, cv2.COLOR_BGR2RGB), (256, 256)).astype(np.float32) / 255.0
        im = np.expand_dims(im, axis =0)
        outcome = [np.argmax(model.predict(im))]
        predictor.extend(list(outcome))
        prob_out = model.predict(im)[0][outcome]
        prob.extend(list(prob_out))
        image_id.extend([i.rsplit("/")[-1]])
        
    final = pd.DataFrame()
    final["id"] = image_id
    final["Label"] = predictor
    final['Probability'] = prob

    final["Label_Name"] = final["Label"].apply(lambda x: labels['CategoryName'][x])
    final.sort_values('id', inplace=True)
    final.to_csv(location+'/'+folder+'.csv',index=False)

In [6]:
for location in locations:
    print(f"Working on location: {location}")
    for folder in folders:
        print(f"Working on folder: {folder}")
        predict_and_save(folder,location)

Working on location: Brinno Camera Test
Working on folder: With Screens
Working on folder: Without Screens
Working on location: Photos to test Inception
Working on folder: With Screens
Working on folder: Without Screens


In [7]:
def predict_and_save_v2(folder, location):
    num_images = len(glob.glob(location+'/'+folder+'/*.jpg'))
    predict_files = glob.glob(location+'/'+folder+'/*.jpg')

    im = cv2.imread(predict_files[0])
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = cv2.resize(im, (256, 256)).astype(np.float32)
    im = np.expand_dims(im, axis = 0)/255
    
    predictor, prob, image_id = [], [], []
    final = pd.DataFrame(columns=['id','prediction','confidence','image'])
    for i in predict_files:
        im = cv2.imread(i)
        im = cv2.resize(cv2.cvtColor(im, cv2.COLOR_BGR2RGB), (256, 256)).astype(np.float32) / 255.0
        im = np.expand_dims(im, axis =0)
        preds = model.predict(im)
        preds = decode_predictions(preds)[0]        
        
        temp_df = pd.DataFrame(preds,columns=['id','prediction','confidence'])
        temp_df['image'] = i.rsplit("/")[-1]
        final = final.append(temp_df)
        
#     final.sort_values('id', inplace=True)
    final.to_csv(location+'/'+folder+'_v2.csv',index=False)

In [8]:
for location in locations:
    print(f"Working on location: {location}")
    for folder in folders:
        print(f"Working on folder: {folder}")
        predict_and_save_v2(folder,location)

Working on location: Brinno Camera Test
Working on folder: With Screens
Working on folder: Without Screens
Working on location: Photos to test Inception
Working on folder: With Screens
Working on folder: Without Screens


In [21]:
from sklearn.metrics import classification_report, confusion_matrix, auc, roc_auc_score, roc_curve

In [26]:
original_test = pd.read_csv('Classification_Report.csv')
brinno_test = pd.read_csv('Classification_Report_Brinno.csv')
no_off_test = pd.read_csv('Classification_Report_NoOffScreens.csv')

In [61]:
print('Original Test Data:\n',
     classification_report(original_test['TRUE'],original_test['Pred']),
     '\nConfusion matrix:\n', confusion_matrix(original_test['TRUE'],original_test['Pred']),
     '\n\nAUC Score: '+str(roc_auc_score(original_test['TRUE'],original_test['Pred'])))


print('\n\nOriginal Without Off Screens Test Data:\n',
     classification_report(no_off_test['TRUE'],no_off_test['Pred']),
     '\nConfusion matrix:\n', confusion_matrix(no_off_test['TRUE'],no_off_test['Pred']),
     '\n\nAUC Score: '+str(roc_auc_score(no_off_test['TRUE'],no_off_test['Pred'])))

print('\n\nBrinno Test Data:\n',
     classification_report(brinno_test['TRUE'],brinno_test['Pred']),
     '\nConfusion matrix:\n', confusion_matrix(brinno_test['TRUE'],brinno_test['Pred']),
     '\n\nAUC Score: '+str(roc_auc_score(brinno_test['TRUE'],brinno_test['Pred'])))


Original Test Data:
               precision    recall  f1-score   support

           0       0.84      0.94      0.89        50
           1       0.93      0.82      0.87        50

   micro avg       0.88      0.88      0.88       100
   macro avg       0.89      0.88      0.88       100
weighted avg       0.89      0.88      0.88       100
 
Confusion matrix:
 [[47  3]
 [ 9 41]] 

AUC Score: 0.8799999999999998


Original Without Off Screens Test Data:
               precision    recall  f1-score   support

           0       0.85      0.94      0.90        50
           1       0.92      0.82      0.87        44

   micro avg       0.88      0.88      0.88        94
   macro avg       0.89      0.88      0.88        94
weighted avg       0.89      0.88      0.88        94
 
Confusion matrix:
 [[47  3]
 [ 8 36]] 

AUC Score: 0.8790909090909091


Brinno Test Data:
               precision    recall  f1-score   support

           0       0.91      0.79      0.84       160
          